In [1]:
!pip install langchain
#!pip install chromadb
!pip install -U langchain-chroma
!pip install pypdf
!pip install pytest
!pip install ollama

In [1]:
import argparse
import os
import shutil
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
#from get_embedding_function import get_embedding_function
from langchain.vectorstores.chroma import Chroma

CHROMA_PATH = "chroma"

In [2]:
document_loader = PyPDFDirectoryLoader("documents")
documents = document_loader.load()
print(documents[1])

page_content='Patients and methods
Patients meeting the inclusion criteria (males, 18-70 years with end-stage renal disease of
unknown or vascular origin) were selected from the REIN® registry and the CRISTAL®
database. Screening on filter papers was performed after patient consent was obtained
during either a dialysis session or a transplantation follow-up visit.
Results
One thousand five hundred and sixty-one end-stage renal disease male patients were
screened and 819 consented (dialysis: n = 242; transplant: n = 577). One single patient was
found with decreased alpha-galactosidase levels <25%. GLA sequencing identified the
p.Phe113Leu variant in favor of an unknown superimposed kidney disease responsible for
end-stage renal disease since this GLA pathogenic variant is associated with a later-onset
cardiac form of Fabry disease with minimal kidney involvement. Family cascade genotyping
revealed a previously undiagnosed affected brother.
Conclusion
The prevalence of Fabry disease in e

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

chunks = text_splitter.split_documents(documents)
print(chunks[2])

page_content='Patients and methods
Patients meeting the inclusion criteria (males, 18-70 years with end-stage renal disease of
unknown or vascular origin) were selected from the REIN® registry and the CRISTAL®
database. Screening on filter papers was performed after patient consent was obtained
during either a dialysis session or a transplantation follow-up visit.
Results
One thousand five hundred and sixty-one end-stage renal disease male patients were
screened and 819 consented (dialysis: n = 242; transplant: n = 577). One single patient was
found with decreased alpha-galactosidase levels <25%. GLA sequencing identified the
p.Phe113Leu variant in favor of an unknown superimposed kidney disease responsible for
end-stage renal disease since this GLA pathogenic variant is associated with a later-onset' metadata={'source': 'documents/Screening for Fabry disease in male patients with end-stage renal disease in western France - ScienceDirect.pdf', 'page': 1}


In [4]:
from langchain_community.embeddings import OllamaEmbeddings

ollama_emb = OllamaEmbeddings(model="llama3")
r2 = ollama_emb.embed_query("Hello.")
r2

[-2.8780405521392822,
 -2.862058162689209,
 1.2840477228164673,
 -0.8955835700035095,
 -2.146989345550537,
 -1.209290862083435,
 0.04406164959073067,
 -0.08449838310480118,
 -2.7292094230651855,
 1.2980396747589111,
 -1.0231959819793701,
 1.1880286931991577,
 -1.2307353019714355,
 -3.0354459285736084,
 -2.843366861343384,
 1.2830970287322998,
 -5.149657726287842,
 -0.3655884861946106,
 -1.386553168296814,
 0.24796156585216522,
 -0.5179820656776428,
 -2.0658469200134277,
 -0.1600186824798584,
 3.7177934646606445,
 -0.5700535774230957,
 -0.7031189203262329,
 0.3961048424243927,
 -1.1706833839416504,
 2.0494823455810547,
 0.931349515914917,
 1.103562831878662,
 2.264519453048706,
 -4.781691074371338,
 3.606318473815918,
 6.046999454498291,
 0.005591519642621279,
 0.15790507197380066,
 1.0134127140045166,
 2.993546962738037,
 2.1877148151397705,
 0.6114796996116638,
 -1.3362579345703125,
 1.6211925745010376,
 1.7027112245559692,
 -1.4447828531265259,
 -1.7308845520019531,
 0.24335438013076

In [17]:
from langchain_chroma import Chroma

db = Chroma(persist_directory=CHROMA_PATH, embedding_function = ollama_emb)
#!chmod 777 chroma/

last_page_id = None
current_chunk_index = 0

for chunk in chunks:
    source = chunk.metadata.get("source")
    page = chunk.metadata.get("page")
    current_page_id = f"{source}:{page}"

    # If the page ID is the same as the last one, increment the index.
    if current_page_id == last_page_id:
        current_chunk_index += 1
    else:
        current_chunk_index = 0

    # Calculate the chunk ID.
    chunk_id = f"{current_page_id}:{current_chunk_index}"
    last_page_id = current_page_id

    # Add it to the page meta-data.
    chunk.metadata["id"] = chunk_id

# Calculate Page IDs.
chunks_with_ids = chunks

# Add or Update the documents.
existing_items = db.get(include=[])  # IDs are always included by default
existing_ids = set(existing_items["ids"])
print(f"Number of existing documents in DB: {len(existing_ids)}")

# Only add documents that don't exist in the DB.
new_chunks = []
for chunk in chunks_with_ids:
    if chunk.metadata["id"] not in existing_ids:
        new_chunks.append(chunk)

if len(new_chunks):
    print(f"👉 Adding new documents: {len(new_chunks)}")
    new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
    db.add_documents(new_chunks, ids=new_chunk_ids)
else:
    print("✅ No new documents to add")

Number of existing documents in DB: 102
👉 Adding new documents: 3934


In [5]:
#CHatGPT
from langchain_chroma import Chroma

db = Chroma(persist_directory=CHROMA_PATH, embedding_function=ollama_emb)

last_page_id = None
current_chunk_index = 0

for chunk in chunks:
    source = chunk.metadata.get("source")
    page = chunk.metadata.get("page")
    current_page_id = f"{source}:{page}"

    # If the page ID is the same as the last one, increment the index.
    if current_page_id == last_page_id:
        current_chunk_index += 1
    else:
        current_chunk_index = 0

    # Calculate the chunk ID.
    chunk_id = f"{current_page_id}:{current_chunk_index}"
    last_page_id = current_page_id

    # Add it to the page meta-data.
    chunk.metadata["id"] = chunk_id

# Calculate Page IDs.
chunks_with_ids = chunks

# Add or Update the documents.
existing_items = db.get(include=[])  # IDs are always included by default
existing_ids = set(existing_items["ids"])
print(f"Number of existing documents in DB: {len(existing_ids)}")

# Only add documents that don't exist in the DB.
new_chunks = [chunk for chunk in chunks_with_ids if chunk.metadata["id"] not in existing_ids]

# Batch processing
BATCH_SIZE = 1000
try:
    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        for start in range(0, len(new_chunks), BATCH_SIZE):
            batch = new_chunks[start:start + BATCH_SIZE]
            batch_ids = [chunk.metadata["id"] for chunk in batch]
            db.add_documents(batch, ids=batch_ids)
        print("✅ New documents added successfully")
    else:
        print("✅ No new documents to add")
except Exception as e:
    print(f"Error occurred during ingestion: {e}")


Number of existing documents in DB: 244
👉 Adding new documents: 630
✅ New documents added successfully


In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

question="Can you relate gen mutations with alfa-galactosidase?"
#expected_response="Holland or Chile."

# Prepare the DB.
#db = Chroma(persist_directory=CHROMA_PATH, embedding_function=ollama_emb)

# Search the DB.
results = db.similarity_search_with_score(question, k=5)

context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=question)
# print(prompt)

model = Ollama(model="llama3")
response_text = model.invoke(prompt)

sources = [doc.metadata.get("id", None) for doc, _score in results]
formatted_response = f"Response: {response_text}\nSources: {sources}"

print(response_text)
print(sources)

Based on the provided context, I can relate gen mutations to alpha-galactosidase.

According to Topaloglu et al. (1999) and Rosenberg et al. (1999), there are 20 novel mutations in the alpha-galactosidase A gene that cause Fabry disease. This suggests a direct link between genetic mutations and alpha-galactosidase, which is an enzyme associated with Fabry disease.

Additionally, Peltola et al. (1994) characterized a point mutation in the aspartylglucosaminidase gene, which is related to alpha-galactosidase, suggesting that genetic mutations can affect the function of this enzyme.

Therefore, it appears that gen mutations are indeed related to alpha-galactosidase, specifically in the context of Fabry disease.
['documents/biomolecules-11-00271.pdf:15:8', 'documents/Human Mutation - 2001 - Blaydon - Fabry disease  20 novel GLA mutations in 35 families.pdf:6:2', 'documents/biomolecules-11-00271.pdf:16:3', 'documents/biomolecules-11-00271.pdf:18:9', 'documents/biomolecules-11-00271.pdf:19:5